In [1]:
# Sequential - initialize our nn as sequence of layers 
# cnn is sequence of layers
from keras.models import Sequential
# 1st step to making cnn, add convolutional layers
# since we're working on images (2d objects), videos are 3d
# 2nd step - add pooling layers in 2d
# 3rd step - flatten to convert pool feature maps thru convo and maxpooling to become feature vector
# 4th step - add connected layers and ann
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# initialize CNN
# making a clf some images if dog or cat
classifier = Sequential()
# cnn is initialized

In [3]:
# step 1 - convolution
# 32,3,3 -> 32 feature detectors, 3 x 3 dimensions 
# input_shape = expected format of our images (colored image 3d, black and white image 2d) 
# (64,64,3) -> 64x64 size image, 3 = color image, 2 = black and white -> number of channels, 
# be careful tensorflow backend and theano backend are different format for above line is different
# use rectify/relu activation function because we need nonlinearity for images 
classifier.add(Convolution2D(32,(3,3), input_shape = (64,64,3), activation = 'relu'))

In [4]:
# step 2 - pooling 
# pooling is used to reduce size of feature map by 2 (divided by 2)
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [5]:
# Add another convolution layer
# at this point we have the pooled feature maps, so we don't need input_shape
# then you can change feature detectors to a different value for increased accuracy 
classifier.add(Convolution2D(32,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [6]:
# step 3 - flatten
# feature map flattening into one vector 
# high numbers represent spatial structure (or specific detail) of input image  
# feature map corresponds to one specific feature of the image, each node that has a high number
classifier.add(Flatten())

In [7]:
# Step 4 - full connection
# number of nodes in hidden layer choose between # of input nodes and # of output nodes
# activate neurons with this activation function 
classifier.add(Dense(units = 128, activation = 'relu'))
# cat or dog is binary outcome (soft max for more than two outcomes)
# expect one node that tells you dog or cat 
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [8]:
# Compiling the CNN 
# adam algo, loss func is binary crossentropy, because classification problem and binary outcome (categorical_crossentropy for more than two)
# performance metric - accuracy is most common one
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
# Part 2 - Fitting the CNN to the images 
# image augmentation allows us to enrich our datasets without adding more images with little or no overfitting
from keras.preprocessing.image import ImageDataGenerator

# image scaling, shearing = geometric transformation, zoom range = random zoom, horizontal_flip = flip images 
train_datagen = ImageDataGenerator(
        #rescale pixel values between 0 or 1, because pixels take value from 0 to 255
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# rescale images of our test set
test_datagen = ImageDataGenerator(rescale=1./255)
# train and valid generator create training and test/validation set 
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        # dimensions expected from CNN input_shape (64,64,3)
        target_size=(64, 64),
        # number of images that will go through CNN
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

# fit cnn onto training set and testing peformance on test set
# steps_per_epoch = no of train samples/batch_size and validation_steps = no of validation samples/batch_size
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000/32,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000/32)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 58s 232ms/step - loss: 0.6563 - acc: 0.5988 - val_loss: 0.6149 - val_acc: 0.6605
Epoch 2/25
250/250 [==============================] - 55s 219ms/step - loss: 0.5777 - acc: 0.6931 - val_loss: 0.5479 - val_acc: 0.7285
Epoch 3/25
250/250 [==============================] - 57s 226ms/step - loss: 0.5376 - acc: 0.7308 - val_loss: 0.5099 - val_acc: 0.7540
Epoch 4/25
250/250 [==============================] - 50s 200ms/step - loss: 0.5131 - acc: 0.7428 - val_loss: 0.5188 - val_acc: 0.7520
Epoch 5/25
250/250 [==============================] - 50s 200ms/step - loss: 0.4916 - acc: 0.7582 - val_loss: 0.4801 - val_acc: 0.7825
Epoch 6/25
250/250 [==============================] - 51s 202ms/step - loss: 0.4680 - acc: 0.7744 - val_loss: 0.4822 - val_acc: 0.7825
Epoch 7/25
250/250 [==============================] - 49s 198ms/step - loss: 0.4539 - acc: 0.7853 - val_lo

In [10]:
# add another convolution layer or/and
# add another fully connected layer to get 80% accuracy 
# insert below after first pooling 